In [1]:
from petastorm import make_reader
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import sys
import desolver.backend as D
from math import isnan
from numba import njit,prange
sys.path.append(os.path.join(os.getcwd(), ".."))
path_formatted_glasgow = "/workspaces/maitrise/data/20221006_physio_quality/set-a/dataParquet"
path_petastorm = f"file:///{path_formatted_glasgow}"


Using numpy backend


In [2]:
##Let's re write each function in a numba compatible version:

@njit
def taux_Mean_fast(signal, taux, hprime, h=0):
    return np.mean((signal[int(h + taux) : int(taux + hprime + h)]))

@njit
def taux_var_fast(signal, taux, hprime, h=0):
    return np.var(signal[int(h + taux) : int(taux + hprime + h)])

@njit(parallel = True)

def adapted_c(c_val,fs,h,hprime,signal):
    for l in c_val:
        if (
            l * fs + hprime * len(signal) + h * len(signal) > len(signal)
            and l * fs + h * len(signal) > len(signal) - 1
        ):
            c = c_val[c_val < l]
            break
    return c



In [3]:
##signal trial

with make_reader(path_petastorm) as reader:
    for sample in reader:
        data = sample
        if data.signal_quality == "acceptable".encode():
            break
        else : 
            pass

ECG_signal = data.signal
ECG_lead = data.signal_names
fs = data.sampling_frequency

dico_ECG = {}

for i,j in zip(ECG_lead,range(12)):
     dico_ECG[i] = ECG_signal[:,j]

print(len(dico_ECG))

/workspaces/maitrise/__pypackages__/3.10/lib/petastorm/fs_utils.py:88: FutureWarning: pyarrow.localfs is deprecated as of 2.0.0, please use pyarrow.fs.LocalFileSystem instead.
  self._filesystem = pyarrow.localfs
/workspaces/maitrise/__pypackages__/3.10/lib/petastorm/etl/dataset_metadata.py:402: FutureWarning: Specifying the 'metadata_nthreads' argument is deprecated as of pyarrow 8.0.0, and the argument will be removed in a future version
  dataset = pq.ParquetDataset(path_or_paths, filesystem=fs, validate_schema=False, metadata_nthreads=10)
/workspaces/maitrise/__pypackages__/3.10/lib/petastorm/etl/dataset_metadata.py:362: FutureWarning: 'ParquetDataset.common_metadata' attribute is deprecated as of pyarrow 5.0.0 and will be removed in a future version.
  if not dataset.common_metadata:
/workspaces/maitrise/__pypackages__/3.10/lib/petastorm/etl/dataset_metadata.py:368: FutureWarning: 'ParquetDataset.common_metadata' attribute is deprecated as of pyarrow 5.0.0 and will be removed in a

12


/workspaces/maitrise/__pypackages__/3.10/lib/petastorm/py_dict_reader_worker.py:146: FutureWarning: 'ParquetDataset.fs' attribute is deprecated as of pyarrow 5.0.0 and will be removed in a future version. Specify 'use_legacy_dataset=False' while constructing the ParquetDataset, and then use the '.filesystem' attribute instead.
  parquet_file = ParquetFile(self._dataset.fs.open(piece.path))
/workspaces/maitrise/__pypackages__/3.10/lib/petastorm/py_dict_reader_worker.py:182: FutureWarning: 'ParquetDataset.partitions' attribute is deprecated as of pyarrow 5.0.0 and will be removed in a future version. Specify 'use_legacy_dataset=False' while constructing the ParquetDataset, and then use the '.partitioning' attribute instead.
  partitions = self._dataset.partitions
/workspaces/maitrise/__pypackages__/3.10/lib/petastorm/py_dict_reader_worker.py:267: FutureWarning: 'ParquetDataset.partitions' attribute is deprecated as of pyarrow 5.0.0 and will be removed in a future version. Specify 'use_le

In [4]:
###Let's give them a try

h = 0.001
hprime = 0.005
sig = dico_ECG[ECG_lead[0]]
c = np.arange(0, (len(sig) / fs) + 0, 1 / fs,dtype = np.float64)
c_t = adapted_c(c,fs,h,hprime,sig.copy())
print(c_t)

[0.000e+00 2.000e-03 4.000e-03 ... 9.984e+00 9.986e+00 9.988e+00]


In [5]:
##Now for the big shot:
@njit
def I1(c, signal, fs, h, hprime, step_c, t0=0):
    tab = np.zeros_like(c,dtype = np.float64)
    for count in prange(len(tab)):
        if count ==0:
            I1c = (
                (1 / (h * len(signal)))
                * step_c
                * np.abs(
                    taux_Mean_fast(signal, t0 * fs, hprime * len(signal), h * len(signal))
                    - taux_Mean_fast(signal, t0 * fs, hprime * len(signal))
                )
            )
            tab[count] = I1c
        else : 
            I1c = tab[count-1]
            I1c = I1c + (
                (1 / (h * len(signal)))
                * step_c
                * np.abs(
                    taux_Mean_fast(signal, t0 * fs, hprime * len(signal), h * len(signal))
                    - taux_Mean_fast(signal, t0 * fs, hprime * len(signal))
                )
            )
            tab[count-1] = I1c
    return tab[:-1]

@njit
def I2(c, signal, fs, h, hprime, step_c, t0=0):
    tab = np.zeros_like(c,dtype = np.float64)
    for count in prange(len(tab)):
        if count ==0:
            I1c = (
                (1 / (h * len(signal)))
                * step_c
                * np.abs(
                    taux_var_fast(signal, t0 * fs, hprime * len(signal), h * len(signal))
                    - taux_var_fast(signal, t0 * fs, hprime * len(signal))
                )
            )
            tab[count] = I1c
        else : 
            I1c = tab[count-1]
            I1c = I1c + (
                (1 / (h * len(signal)))
                * step_c
                * np.abs(
                    taux_var_fast(signal, t0 * fs, hprime * len(signal), h * len(signal))
                    - taux_var_fast(signal, t0 * fs, hprime * len(signal))
                )
            )
            tab[count-1] = I1c
    return tab[:-1]

In [6]:
##Trial : 

I1_t = I2(c_t,sig,fs,h,hprime,1/fs)
print(I1_t)

[7.42399716e-09 3.71199858e-09 3.71199858e-09 ... 3.71199858e-09
 3.71199858e-09 3.71199858e-09]


In [7]:
###Now let's create the main runner : 
@njit
def discrepancies_mean_curve(signal_tot, fs, h, hprime, step, t0=0):
    c1 = np.arange(t0, (len(signal_tot) / fs) + t0, step,dtype = np.float64)
    c_adapted = adapted_c(c1,fs,h,hprime,signal_tot)
    I1_t = I1(c_adapted, signal_tot, fs, h, hprime, step, t0)
    I2_t = I2(c_adapted, signal_tot, fs, h, hprime, step, t0)
    return I1_t, I2_t, c_adapted

In [8]:
##Test : 

I1_test,I2_test,c = discrepancies_mean_curve(sig,fs,h,hprime,1/fs)

In [9]:
##Der interval calculator for each lead 
@njit
def Interval_calculator_lead_fast(signal, fs, t0=0):
    h = 0.001
    hprime = 0.005
    c1 = np.arange(t0, (len(signal) / fs) + t0, 1/fs)
    c_adapted = adapted_c(c1,fs,h,hprime,signal)
    I1c = I1(c_adapted, signal, fs, h, hprime, 1/fs, t0)
    I2c = I2(c_adapted, signal, fs, h, hprime, 1/fs, t0)
    c1 = c[I1c[~np.isnan(I1c)]==np.max(I1c[~np.isnan(I1c)]) / 2]
    c2 = c[I2c[~np.isnan(I2c)]==np.max(I2c[~np.isnan(I2c)]) / 2]
    cs = np.minimum(np.mean(c1), np.mean(c2))
    dic_segment_lead = (cs - t0) * fs
    if isnan(dic_segment_lead) or dic_segment_lead<100 :
        dic_segment_lead = 2500
    return dic_segment_lead

In [10]:
##Test : 
optimal_test = Interval_calculator_lead_fast(sig,fs)
print(optimal_test)

2496.9999999999995


In [11]:
##Now for all The lead :

def Interval_calculator_all(dico_signal, name_signal, fs):
    dic_segment_lead = {}
    for i in name_signal:
        dic_segment_lead[i] = int(Interval_calculator_lead_fast(dico_signal[i], fs))
    return dic_segment_lead

In [12]:
##Test : 
dic_segment_lead = Interval_calculator_all(dico_ECG,ECG_lead,fs)
print(dic_segment_lead)

{b'I': 2496, b'II': 2496, b'III': 2496, b'aVR': 2496, b'aVF': 2496, b'aVL': 2496, b'V1': 2496, b'V2': 2496, b'V3': 2496, b'V4': 2496, b'V5': 2496, b'V6': 2496}


In [13]:
##Now the element of the TSD:

from numpy import int64


@njit
def Lm_q(signal1, m, k, fs):
    N = len(signal1)
    n = np.floor((N - m) / k)
    norm = (N - 1) / (n * k * (1 / fs))
    #sum = np.sum(np.abs(np.diff(signal1[m::k], n=1)))
    sum = 0
    for i in prange(1,n):
        sum = sum + np.absolute(signal1[m+i*k]-signal1[m+(i-1)*k])
    Lmq = (sum * norm) / k
    return Lmq
@njit
def Lq_k(signal, k, fs):
    #calc_L_series = np.frompyfunc(lambda m: Lm_q(signal, m, k, fs), 1, 1)
    calc_L_series = np.zeros(k)
    for m in prange(1,k+1):
        calc_L_series[m-1] = Lm_q(signal,m,k,fs)
    L_average = np.mean(calc_L_series)
    return L_average

In [14]:
###THE true challenge : 
@njit
def TSD_mean_calculator_fast(signal2,segment_length,fs):
    Ds = np.zeros(int(len(signal2)-segment_length))
    for w in prange(1,int(len(signal2)-segment_length)):
        sig_true = signal2[int((w - 1)): int((w)+segment_length)]
        L1 = Lq_k(sig_true, 1, fs)
        L2 = Lq_k(sig_true,2,fs)
        Ds[w] = (np.log(L1) - np.log(L2)) / (np.log(2))
    return np.mean(Ds), np.std(Ds)

    

In [15]:
##Test :
val,_ = TSD_mean_calculator_fast(sig,2500,fs)
print(val)

##Comparison : 
def TSD_mean_calculator(signal,segment_length,fs,k=1):

    X = np.c_[[signal[int((w - 1)): int((w)+segment_length)] for w in range(1, int(len(signal)-segment_length),k)]]
    L1 = np.array([Lq_k(X[i, :], 1, fs) for i in range(X.shape[0])])
    L2 = np.array([Lq_k(X[i, :], 2,fs) for i in range(X.shape[0])])
    Ds = (np.log(L1) - np.log(L2)) / (np.log(2))
    return np.mean(Ds), np.std(Ds)
val_c,_ = TSD_mean_calculator(sig,2500,fs)
print(val_c)



1.3661486613441125
1.3666953394799033


In [16]:
###THE UTILMATE TEST NIGH!!! THIS TEST, IF SUCCEEDED, WILL RULE THEM ALLLLLLLLL MWAHAHAHHAHAHAHAHAHAHHAHHAHAHAHAHA :

def is_segment_flatline(sig):
    cond = np.where(np.diff(sig.copy(),1) != 0.0, False, True)
    if len(cond[cond == True]) < 0.70 * len(sig):
        return False
    return True

def TSD_index_fast(dico_signal, name_lead, fs):

    ###Index Creation :TSD
    ###The label will be as follow : mean(TSD) < 1.25 = Acceptable;mean(SDR of all lead) >1.25 = Unacceptable
    ##For each lead, we will return a more precise classification based on the folloying rules:
    ## TSD<1.25 = Good quality ; 1.25<TSD<1.40 = Medium quality; TSD>1.4 = Bad quality
    # dico_seg = Interval_calculator(dico_signal,name_lead,fs,t0)
    dico_D = {}
    D_arr = np.array([])
    dic_segment = Interval_calculator_all(dico_signal,name_lead,fs)
    #dic_segment = 2500
    for i in name_lead:
        if is_segment_flatline(dico_signal[i]):
            dico_D[i] = (2,dico_signal[i])
            D_arr = np.append(D_arr,2)
        else :
            Dv, _ = TSD_mean_calculator_fast(dico_signal[i],dic_segment[i],fs)
            dico_D[i] = (Dv, dico_signal[i])
            D_arr = np.append(D_arr, Dv)
    return dico_D, np.mean(D_arr)

In [17]:
## AND NOW THE TEST:

dico_d,val = TSD_index_fast(dico_ECG,ECG_lead,fs)
print(dico_d)

{b'I': (1.3661558656318777, array([-0.02 , -0.015, -0.01 , ...,  0.42 ,  0.47 ,  0.5  ], dtype=float32)), b'II': (1.3348961700832476, array([-0.06, -0.06, -0.07, ...,  0.28,  0.4 ,  0.54], dtype=float32)), b'III': (1.4068697738421014, array([-0.04 , -0.045, -0.06 , ..., -0.14 , -0.07 ,  0.04 ], dtype=float32)), b'aVR': (1.371269766024023, array([ 0.04 ,  0.035,  0.04 , ..., -0.35 , -0.435, -0.52 ], dtype=float32)), b'aVF': (1.4059469093817505, array([-0.05 , -0.05 , -0.065, ...,  0.07 ,  0.165,  0.29 ], dtype=float32)), b'aVL': (1.4283963160087998, array([0.01 , 0.015, 0.025, ..., 0.28 , 0.27 , 0.23 ], dtype=float32)), b'V1': (1.4604248968016966, array([ 0.06 ,  0.07 ,  0.075, ..., -0.055, -0.12 , -0.2  ], dtype=float32)), b'V2': (1.1890368473767963, array([-0.06 , -0.07 , -0.07 , ...,  1.075,  1.035,  0.94 ], dtype=float32)), b'V3': (1.2198807830096552, array([-0.04 , -0.05 , -0.055, ...,  1.06 ,  1.08 ,  1.06 ], dtype=float32)), b'V4': (1.2076823561799914, array([-0.04 , -0.05 , -0.0